In [236]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.feature_selection import SelectKBest, f_regression

In [237]:
df = pd.read_csv('../../datasets/processed_data/combined_features/IPCALAB.BO.csv')

In [238]:
df.columns

Index(['date', 'open', 'close', 'adj close', 'volume', 'low', 'high',
       'volume_adi', 'volume_obv', 'volume_cmf',
       ...
       'inr=x_percent_change', 'cl=f_percent_change',
       'treasury_yeild_10_years_percent_change', 'usdx-index_percent_change',
       '^nsei_percent_change', '^bsesn_percent_change', '^gspc_percent_change',
       'hsi_percent_change', 'sha_percent_change', '^sti_percent_change'],
      dtype='object', length=635)

In [239]:
def create_custom_target(df: pd.DataFrame) -> pd.DataFrame:
    """
    This function creates the custom target price, which is computed as ln(high/yesterday_close)
    """    
    # make a copy of the dataframe so as not to change the original dataframe
    data_df = df.copy()
    
    # create yesterday_close feature as
    data_df['yesterday_close'] = data_df['close'].shift(1)
    
    # create custom target price to predict, computing  ln(high/yesterday_close)
    data_df['ln_target'] = np.log(data_df['high'] / data_df['yesterday_close'])
    
    # as yesterday close would not be available for first day, 
    # we would not have custom target price for that day, which needs to be excluded 
    return(data_df.iloc[1:, ])

In [240]:
df = create_custom_target(df)

In [241]:
df['yesterday_close']

1        58.779999
2        59.625000
3        60.000000
4        58.235001
5        57.294998
           ...    
3685    689.049988
3686    684.150024
3687    682.250000
3688    683.599976
3689    687.250000
Name: yesterday_close, Length: 3689, dtype: float64

In [242]:
df['ln_target'] 

1       0.020626
2       0.029335
3       0.021435
4       0.023169
5       0.003572
          ...   
3685    0.002030
3686    0.005175
3687    0.004752
3688    0.014450
3689    0.028333
Name: ln_target, Length: 3689, dtype: float64

In [243]:
# Drop cols with NaN values
df = df.dropna(axis=1, how='all')
df.fillna(method='ffill', inplace=True)

In [244]:
combined_date_df = df['date']
train_date, test_date = train_test_split(combined_date_df, train_size=0.8, shuffle=False)

In [245]:
df_without_date = df.drop(columns=["date"])

In [246]:
df_without_date.dropna(inplace=True)

In [247]:
def timeseries_to_supervise(df, window_size, target):    
    X = []
    y = []
    indx = []
    no_records = len(df)
    #     
    for i in range(window_size, no_records):
        X.append(df.iloc[i-window_size:i].drop(target, axis=1).values.flatten())  # Collect past records as a sequence
        y.append(df.iloc[i][target])  # Next record as target variable
        indx.append(np.arange(i-window_size, i))

    X = pd.DataFrame(X)
    y = pd.Series(y)
    return(X, y, indx)

In [248]:
window_size = 10

In [249]:
# do train/test split the data with shuffle = False
train_data, test_data = train_test_split(df_without_date, train_size=0.8, shuffle=False)

# Identify the indices of the specified sentiment columns in the original dataset
sentiment_columns = ['agg_polarity', 'agg_compound', 'topic_polarity', 'topic_compound', 'ticker_polarity', 'ticker_compound']
column_indices = [train_data.columns.get_loc(col) for col in sentiment_columns if col in train_data.columns]

column_indices

[616, 618, 619, 621, 623, 622]

In [250]:
# convert timeseries to be used in supervise learning model
X_train, y_train, indx_train = timeseries_to_supervise(train_data, window_size, 'ln_target')  

# convert timeseries to be used in supervise learning model    
X_test, y_test, indx_test = timeseries_to_supervise(test_data, window_size, 'ln_target')  

In [251]:
# Use SelectKBest to select the top 44 features
selector = SelectKBest(score_func=f_regression, k=44)
X_train_44 = selector.fit_transform(X_train, y_train)
X_test_44 = selector.transform(X_test)

# Determine if the specified columns are among the top 44 features
selected_indices = selector.get_support(indices=True)
missing_indices = [idx for idx in column_indices if idx not in selected_indices]

# Add the specified columns if they are not among the top 44 features
for idx in missing_indices:
    extracted_column_train = X_train.iloc[:, idx].values.reshape(-1, 1)
    extracted_column_test = X_test.iloc[:, idx].values.reshape(-1, 1)
    X_train_44 = np.hstack([X_train_44, extracted_column_train])
    X_test_44 = np.hstack([X_test_44, extracted_column_test])

# If some of the specified columns were already in the top 44, 
# select additional top features to make the total count 50
remaining_indices = [i for i in range(X_train.shape[1]) if i not in selected_indices and i not in missing_indices]
num_additional_features_needed = 50 - X_train_44.shape[1]

if num_additional_features_needed > 0:
    additional_selector = SelectKBest(score_func=f_regression, k=num_additional_features_needed)
    additional_selector.fit(X_train.iloc[:, remaining_indices], y_train)
    X_train_additional = additional_selector.transform(X_train.iloc[:, remaining_indices])
    X_test_additional = additional_selector.transform(X_test.iloc[:, remaining_indices])

    X_train_44 = np.hstack([X_train_44, X_train_additional])
    X_test_44 = np.hstack([X_test_44, X_test_additional])

X_train_50 = X_train_44
X_test_50 = X_test_44

In [252]:
rf_regressor = RandomForestRegressor(
    n_estimators=100,
    min_samples_split=2,
    min_samples_leaf=2,
    bootstrap=True
)
rf_regressor.fit(X_train_50, y_train)

RandomForestRegressor(min_samples_leaf=2)

In [253]:
def convert_custom_target_to_actual_for_supervise(df: pd.DataFrame, window: int, y: "pd.Series[int]") -> "pd.Series[int]":
    """
    this module converts custom target - ln(high/yesterday_close) to actual high price again for timeseries converted data using rolling         window of size 10
    """
    data_df = df.copy()
    
    # exclude first 10 rows of train/test data, as while us
    
    y = np.exp(y) * data_df.loc[data_df.index[window:], 'yesterday_close'].reset_index(drop=True)
    return(y)    

In [254]:
def evaluate_model(model, window, dev_data, dev_date, X_test, y_test):
    
    # do target prediction using the provide model
    y_pred = model.predict(X_test)

    # convert back to original value, before computing mape            
    y_test = convert_custom_target_to_actual_for_supervise(dev_data, window, y_test)
    y_pred = convert_custom_target_to_actual_for_supervise(dev_data, window, y_pred)

    dev_dates = dev_date[window:].reset_index(drop=True)
    predictions_df = pd.DataFrame({'date': dev_dates, 'y_test': y_test, 'y_pred': y_pred})

    # compute regression metric - mape 
    mape = mean_absolute_percentage_error(y_test, y_pred)

    # compute rmse metric
    rmse = mean_squared_error(y_test, y_pred, squared=False)        
    return(predictions_df, mape, rmse)

In [255]:
predictions_df, mape, rmse = evaluate_model(rf_regressor, window_size, test_data, test_date, X_test_50, y_test)


In [256]:
predictions_df, mape, rmse

(           date      y_test      y_pred
 0    2020-06-23  837.500000  846.034409
 1    2020-06-24  836.750000  848.152623
 2    2020-06-25  842.500000  842.960338
 3    2020-06-26  854.575012  850.550175
 4    2020-06-29  853.799988  864.928157
 ..          ...         ...         ...
 723  2023-05-24  690.450012  712.251084
 724  2023-05-25  687.700012  732.464890
 725  2023-05-26  685.500000  732.625586
 726  2023-05-30  693.549988  733.668758
 727  2023-05-31  707.000000  738.255725
 
 [728 rows x 3 columns],
 0.031765635812092144,
 44.19230953769315)

In [257]:
print(f"MAPE for model with top 50 features: {mape*100:.2f}%")

MAPE for model with top 50 features: 3.18%


In [258]:
predictions_df.to_csv("rf_IPCALAB.csv", index=False)

In [188]:
import altair as alt

alt.themes.enable('fivethirtyeight')

predictions_df['date'] = pd.to_datetime(predictions_df['date'])

predictions_df['label'] = 'Actual'
predictions_df['predicted_label'] = 'Predicted'

# Actual high price line
line1 = alt.Chart(predictions_df).mark_line(strokeWidth=2).encode(
    x='date:T',
    y=alt.Y('y_test:Q', title='Price', scale=alt.Scale(zero=False)),
    color=alt.Color('label:N', legend=alt.Legend(title="Line Type")),
    tooltip=['date', 'y_test', 'y_pred']
)

# Predicted high price line
line2 = alt.Chart(predictions_df).mark_line(strokeWidth=1, strokeDash=[3, 3]).encode(
    x='date:T',
    y=alt.Y('y_pred:Q', title='', scale=alt.Scale(zero=False)),
    color=alt.Color('predicted_label:N', legend=alt.Legend(title="Line Type")),
    tooltip=['date', 'y_test', 'y_pred']
)

# Combine the two lines
chart = alt.layer(line1, line2).properties(
    title='Actual vs Predicted High Prices',
    width=650,
    height=400
).interactive()

chart

alt.LayerChart(...)

Repeat the process for the top 50 features without sentiment of IPCALAB.BO

In [189]:
df = pd.read_csv('../../datasets/processed_data/combined_features/IPCALAB.BO.csv')

In [190]:
df_without_sentiment = df.drop(columns=['agg_polarity', 'agg_compound', 'topic_polarity', 'topic_compound', 'ticker_polarity', 'ticker_compound'])

In [191]:
df_without_sentiment = create_custom_target(df_without_sentiment)

In [192]:
# Drop cols with NaN values
df_without_sentiment = df_without_sentiment.dropna(axis=1, how='all')
df_without_sentiment.fillna(method='ffill', inplace=True)

In [193]:
combined_date_df = df_without_sentiment['date']
train_date, test_date = train_test_split(combined_date_df, train_size=0.8, shuffle=False)

In [194]:
df_without_date = df_without_sentiment.drop(columns=["date"])

In [195]:
df_without_date.dropna(inplace=True)

In [196]:
# do train/test split the data with shuffle = False
train_data, test_data = train_test_split(df_without_date, train_size=0.8, shuffle=False)

# convert timeseries to be used in supervise learning model
X_train, y_train, indx_train = timeseries_to_supervise(train_data, window_size, 'ln_target')  

# convert timeseries to be used in supervise learning model    
X_test, y_test, indx_test = timeseries_to_supervise(test_data, window_size, 'ln_target')  

In [197]:
selector = SelectKBest(score_func=f_regression, k=50)
X_train_50 = selector.fit_transform(X_train, y_train)
X_test_50 = selector.transform(X_test)

In [198]:
# Train the model on the top 50 features
model_50 = RandomForestRegressor(
    n_estimators=200,
    min_samples_split=2,
    min_samples_leaf=2,
    bootstrap=True
)
model_50.fit(X_train_50, y_train)

RandomForestRegressor(min_samples_leaf=2, n_estimators=200)

In [199]:
predictions_df, mape, rmse = evaluate_model(model_50, window_size, test_data, test_date, X_test_50, y_test)

In [200]:
predictions_df, mape, rmse

(           date      y_test      y_pred
 0    2020-06-23  837.500000  844.771408
 1    2020-06-24  836.750000  856.059889
 2    2020-06-25  842.500000  839.776096
 3    2020-06-26  854.575012  850.103536
 4    2020-06-29  853.799988  865.331371
 ..          ...         ...         ...
 723  2023-05-24  690.450012  715.429895
 724  2023-05-25  687.700012  730.198732
 725  2023-05-26  685.500000  729.942322
 726  2023-05-30  693.549988  729.320282
 727  2023-05-31  707.000000  734.061317
 
 [728 rows x 3 columns],
 0.03219285241111322,
 43.81502233545428)

In [201]:
print(f"MAPE for model with top 50 features without sentiment features: {mape*100:.2f}%")

MAPE for model with top 50 features without sentiment features: 3.22%


In [202]:
import altair as alt

alt.themes.enable('fivethirtyeight')

predictions_df['date'] = pd.to_datetime(predictions_df['date'])

predictions_df['label'] = 'Actual'
predictions_df['predicted_label'] = 'Predicted'

# Actual high price line
line1 = alt.Chart(predictions_df).mark_line(strokeWidth=2).encode(
    x='date:T',
    y=alt.Y('y_test:Q', title='Price', scale=alt.Scale(zero=False)),
    color=alt.Color('label:N', legend=alt.Legend(title="Line Type")),
    tooltip=['date', 'y_test', 'y_pred']
)

# Predicted high price line
line2 = alt.Chart(predictions_df).mark_line(strokeWidth=1, strokeDash=[3, 3]).encode(
    x='date:T',
    y=alt.Y('y_pred:Q', title='', scale=alt.Scale(zero=False)),
    color=alt.Color('predicted_label:N', legend=alt.Legend(title="Line Type")),
    tooltip=['date', 'y_test', 'y_pred']
)

# Combine the two lines
chart = alt.layer(line1, line2).properties(
    title='Actual vs Predicted High Prices',
    width=650,
    height=400
).interactive()

chart

alt.LayerChart(...)